# Import packages

In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load MD trajectories

filename should be the location where the numpy array is stored 

In [7]:
# load cartesian coordinate file. 
file_name = '/Users/user/Mirror/long_seq_LSTM/extracted_pdb_trajectory_more_config_recentering_aligned.npy'
full_data = np.load(file_name)[:,1:]

In [12]:
# shape of the data 
full_data.shape, 'number of atoms: ', full_data.shape[1]/3


((5856, 6546), 'number of atoms: ', 2182.0)

# Select the training set and normalise 

In [13]:
training_set = full_data[:3000,:]
training_set.shape

(3000, 6546)

Reshape the data to 2D and then normalise the three axis together

In [15]:
num_samples, num_features = training_set.shape
data_2d = training_set.reshape(num_samples * num_features, 1)



scaler = MinMaxScaler()
scaler.fit(data_2d)

data_2d = scaler.transform(data_2d)


training_set = data_2d.reshape(num_samples, num_features)


training_set.shape, f'max value: {training_set.max()} and the min valu {training_set.min()}'

((3000, 6546), 'max value: 0.9999999999999998 and the min valu 0.0')

## Save the scaler to rescale the predictions later 

# Convert the data into timeSeries data 